# Testing Custom Timeouts


## Prerequistes

You will need

 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - [Helm](https://github.com/kubernetes/helm)
 - [Minikube](https://github.com/kubernetes/minikube) version v0.24.0 or greater
 - [seldon-core Python package](https://pypi.org/project/seldon-core/) (```pip install seldon-core```)


## Create Cluster

Start minikube and ensure custom resource validation is activated and there is 5G of memory. 

Your start command with the kvm driver would then look like:
```
minikube start --vm-driver kvm2 --memory 4096
```

## Setup

In [1]:
!kubectl create namespace seldon

namespace/seldon created


In [2]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "minikube" modified.


In [3]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding.rbac.authorization.k8s.io/kube-system-cluster-admin created


## Install Helm

In [4]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

serviceaccount/tiller created
clusterrolebinding.rbac.authorization.k8s.io/tiller created
$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


In [5]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

Waiting for deployment "tiller-deploy" rollout to finish: 0 of 1 updated replicas are available...
deployment "tiller-deploy" successfully rolled out


## Start seldon-core

Install the custom resource definition

In [6]:
!helm install ../helm-charts/seldon-core-crd --name seldon-core-crd --set usage_metrics.enabled=true

NAME:   seldon-core-crd
LAST DEPLOYED: Sat Feb 23 11:05:26 2019
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/Deployment
NAME                        DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-spartakus-volunteer  1        1        1           0          0s

==> v1/ServiceAccount
NAME                        SECRETS  AGE
seldon-spartakus-volunteer  1        0s

==> v1beta1/ClusterRole
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1beta1/ClusterRoleBinding
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1/Pod(related)
NAME                                         READY  STATUS             RESTARTS  AGE
seldon-spartakus-volunteer-5554c4d8b6-d9zqz  0/1    ContainerCreating  0         0s

==> v1/ConfigMap
NAME                     DATA  AGE
seldon-spartakus-config  3     4s

==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  0s


NOTES:
NOTES: TODO



In [7]:
!helm install ../helm-charts/seldon-core --name seldon-core --namespace seldon \
    --set ambassador.enabled=true \
    --set ambassador.env.AMBASSADOR_SINGLE_NAMESPACE=true

NAME:   seldon-core
LAST DEPLOYED: Sat Feb 23 11:05:36 2019
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1/Role
NAME          AGE
ambassador    0s
seldon-local  0s

==> v1/RoleBinding
NAME        AGE
seldon      0s
ambassador  0s

==> v1/Service
NAME                          TYPE       CLUSTER-IP      EXTERNAL-IP  PORT(S)                        AGE
seldon-core-ambassador        NodePort   10.101.143.247  <none>       80:30876/TCP,443:32706/TCP     0s
seldon-core-ambassador-admin  NodePort   10.98.231.238   <none>       8877:31259/TCP                 0s
seldon-core-seldon-apiserver  NodePort   10.101.181.168  <none>       8080:31540/TCP,5000:32075/TCP  0s
seldon-core-redis             ClusterIP  10.102.240.177  <none>       6379/TCP                       0s

==> v1beta1/Deployment
NAME                                DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-core-ambassador              1        1        1           0          0s
seldon-core-seldon-apiserver        1    

Check all services are running before proceeding.

In [8]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager
!kubectl rollout status deploy/seldon-core-seldon-apiserver
!kubectl rollout status deploy/seldon-core-ambassador 

Waiting for deployment "seldon-core-seldon-cluster-manager" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-core-seldon-cluster-manager" successfully rolled out
deployment "seldon-core-seldon-apiserver" successfully rolled out
Waiting for deployment "seldon-core-ambassador" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-core-ambassador" successfully rolled out


## Port forward to Ambassador


**Ensure you port forward ambassador**:

```
kubectl port-forward $(kubectl get pods -n seldon -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
```

Illustration of both REST and gRPC requests. 

In [9]:
!pygmentize resources/model_short_timeouts.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "seldon-deployment-example"
    },
    "spec": {
        "annotations": {
            "project_name": "FX Market Prediction",
            "deployment_version": "v1",
	    "seldon.io/rest-read-timeout":"1"
        },
        "name": "test-deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mock_classifier:1.0",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier",
                                "resources": {
                                    "requests": {
                          

## Seldon Deployment with Short Timeout

Deploy the runtime graph to kubernetes.

In [10]:
!kubectl apply -f resources/model_short_timeouts.json -n seldon

seldondeployment.machinelearning.seldon.io/seldon-deployment-example created


In [11]:
!kubectl rollout status deploy/test-deployment-fx-market-predictor-7cd068f

Waiting for deployment "test-deployment-fx-market-predictor-7cd068f" rollout to finish: 0 of 1 updated replicas are available...
deployment "test-deployment-fx-market-predictor-7cd068f" successfully rolled out


In [12]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="seldon-deployment-example",namespace="seldon")

This next request should fail as the timeout is too short

In [14]:
r = sc.predict(gateway="ambassador",transport="rest")
print(r)

Success:False message:504:Gateway Timeout
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.3908024686100948
  }
}

Response:
None


Delete this graph and recreate one with a longer timeout

In [15]:
!kubectl delete -f resources/model_short_timeouts.json

seldondeployment.machinelearning.seldon.io "seldon-deployment-example" deleted


## Seldon Deployment with Long Timeout

In [16]:
!pygmentize resources/model_long_timeouts.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "model-long-timeout"
    },
    "spec": {
        "annotations": {
            "deployment_version": "v1",
	    "seldon.io/rest-read-timeout":"100000",
	    "seldon.io/rest-connection-timeout":"100000",	    
	    "seldon.io/grpc-read-timeout":"100000"
        },
        "name": "long-to",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mock_classifier:1.0",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier",
                                "resources": {
                                    "

In [17]:
!kubectl apply -f resources/model_long_timeouts.json -n seldon

seldondeployment.machinelearning.seldon.io/model-long-timeout created


In [18]:
!kubectl rollout status deploy/long-to-test-7cd068f

deployment "long-to-test-7cd068f" successfully rolled out


This next request should work as the timeout is much longer

In [19]:
r = sc.predict(gateway="ambassador",transport="rest",deployment_name="model-long-timeout")
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.011317539951789746
  }
}

Response:
meta {
  puid: "scdgs6ova24be3f5r07cv3fuut"
  requestPath {
    key: "classifier"
    value: "seldonio/mock_classifier:1.0"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.05188976922442161
  }
}

